https://stackoverflow.com/questions/73095228/xarray-cannot-open-url

# this does  work on https://pangeo-clivar.vm.fedcloud.eu/jupyterhub/

Jupyterlab have 8G of RAM

In [9]:
import numpy as np
model_fnames_dict = np.load("models.npy", allow_pickle=True).item()
opendap=model_fnames_dict['CMCC-CM2-HR4']
opendap[7]

'http://esgf-data1.llnl.gov/thredds/dodsC/css03_data/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc'

In [10]:
import xarray as xr
ds = xr.open_mfdataset([opendap[7]], chunks={'time': 10, 'i' : 200, 'j': 200})
ds

OSError: [Errno -90] NetCDF: file not found: b'http://esgf-data1.llnl.gov/thredds/dodsC/css03_data/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc'

In [1]:
from pyesgf.logon import LogonManager
lm = LogonManager()
lm.is_logged_on()
#hostname='esgf-node.llnl.gov'
hostname='esgf-node.ipsl.upmc.fr'
lm.logon(hostname=hostname, interactive=True, bootstrap=True, username='tinaok', password='Clivar-2022')
lm.is_logged_on()

True

In [2]:
from pyesgf.search import SearchConnection
server='https://esgf-data.dkrz.de/esg-search'
#server='https://esgf-data.llnl.gov/esg-search'
#server='https://esgf-data.ipsl.upmc.fr/esg-search'
#server='https://esgf-data.ceda.ac.uk/esg-search'


conn = SearchConnection(server, distrib=True)

For 'CMCC-CM2-HR4', data_node='esgf.ceda.ac.uk' works.  The one from  'http://esgf-data1.llnl.gov/ and 'http://esgf-node2.cmcc.it/ does not work.



In [8]:
#http://esgf-data1.llnl.gov/thredds/dodsC/css03_data/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc
experiment_id='historical',


source_id='CMCC-CM2-HR4'
activity_id='OMIP'
experiment_id='omip2'
variable_id='vmo'

data_node='esgf.ceda.ac.uk'
#ctx = conn.new_context(facets='variable,source_id,frequency',#,data_node',
ctx = conn.new_context(facets='variable,source_id,frequency,experiment_id,data_node',
#    project='CMIP6',
    source_id=source_id,
   experiment_id=experiment_id,
    variable=variable_id,
    frequency='mon',
#    variant_label='r1i1p1f2',
    data_node=data_node
)
print(ctx.hit_count)
urls=[]
opendaps=[]
for i in range(ctx.hit_count):
    print(i)
    result = ctx.search()[i]
    files = result.file_context().search()
    for file in files:
        urls.append(file.urls['HTTPServer'][0][0])
        opendaps.append(file.opendap_url)

1
0



-------------------------------------------------------------------------------
Warning - defaulting to search with facets=*

This behavior is kept for backward-compatibility, but ESGF indexes might not
successfully perform a distributed search when this option is used, so some
results may be missing.  For full results, it is recommended to pass a list of
facets of interest when instantiating a context object.  For example,

      ctx = conn.new_context(facets='project,experiment_id')

Only the facets that you specify will be present in the facets_counts dictionary.

This warning is displayed when a distributed search is performed while using the
facets=* default, a maximum of once per context object.  To suppress this warning,
set the environment variable ESGF_PYCLIENT_NO_FACETS_STAR_WARNING to any value
or explicitly use  conn.new_context(facets='*')

-------------------------------------------------------------------------------


In [11]:
import xarray as xr
ds = xr.open_mfdataset([opendaps[-1]], chunks={'time': 10, 'i' : 200, 'j': 200})
ds

<xarray.Dataset>
Dimensions:             (time: 132, bnds: 2, lev: 50, j: 1050, i: 1440,
                         vertices: 4)
Coordinates:
  * time                (time) object 2008-01-16 12:00:00 ... 2018-12-16 12:0...
  * lev                 (lev) float64 0.5126 1.621 2.858 ... 5.498e+03 5.904e+03
  * j                   (j) int32 0 1 2 3 4 5 ... 1044 1045 1046 1047 1048 1049
  * i                   (i) int32 0 1 2 3 4 5 ... 1434 1435 1436 1437 1438 1439
    latitude            (j, i) float64 dask.array<chunksize=(200, 200), meta=np.ndarray>
    longitude           (j, i) float64 dask.array<chunksize=(200, 200), meta=np.ndarray>
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds           (time, bnds) object dask.array<chunksize=(10, 2), meta=np.ndarray>
    lev_bnds            (lev, bnds) float64 dask.array<chunksize=(50, 2), meta=np.ndarray>
    vertices_latitude   (j, i, vertices) float64 dask.array<chunksize=(200, 200, 4), meta=np.ndarray>
    vertices_longitude  (j, i, vertices) float64 dask.array<chunksize=(200, 200, 4), meta=np.ndarray>
    vmo                 (time, lev, j, i) float32 dask.array<chunksize=(10, 50, 200, 200), meta=np.ndarray>
Attributes: (12/40)
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     OMIP
    comment:                         Ocean initial conditions: WOA 2013 T & S...
    contact:                         Pier Giuseppe Fogli (piergiuseppe.fogli@...
    creation_date:                   2020-01-31T21:19:12Z
    data_specs_version:              01.00.31
    ...                              ...
    variant_label:                   r1i1p1f1
    license:                         CMIP6 model data produced by CMCC is lic...
    cmor_version:                    3.5.0
    _NCProperties:                   version=2,netcdf=4.6.2,hdf5=1.10.5
    tracking_id:                     hdl:21.14100/a54c0833-3909-4ed4-b5eb-e57...
    DODS_EXTRA.Unlimited_Dimension:  time

In [12]:
import xarray as xr
ds = xr.open_mfdataset([urls[-1]], chunks={'time': 10, 'i' : 200, 'j': 200})
ds

syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <html^><head><title>404 Not Found</title></head><body><center><h1>404 Not Found</h1></center><hr><center>nginx</center></body></html>


OSError: [Errno -90] NetCDF: file not found: b'https://esgf.ceda.ac.uk/thredds/fileServer/esg_cmip6/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc'

In [14]:
# it crashed for opening, probably i missed something, but wget works so http access is working.
!wget https://esgf.ceda.ac.uk/thredds/fileServer/esg_cmip6/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc


--2022-10-21 13:54:45--  https://esgf.ceda.ac.uk/thredds/fileServer/esg_cmip6/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc
Resolving esgf.ceda.ac.uk (esgf.ceda.ac.uk)... 130.246.128.97
Connecting to esgf.ceda.ac.uk (esgf.ceda.ac.uk)|130.246.128.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17047143485 (16G) [application/octet-stream]
Saving to: ‘vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc.2’

CC-CM2-HR4_omip2_r1   0%[                    ]   9.80M  1.17MB/s    eta 2h 13m ^C


## Access from opendap or https both works.

If using opendap in paralle (like let dask to access each file, and cut slice, and save that part in netcdf or zarr, using dask delay for example) is too much for ESGF's opendap server, using kerchunk to analyse https file, and then use that as zarr might be a solution (before that you'll need to check the NetCDF file is HDF5  and have chunk as you wish to use or not) 